In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    NormalizedString,
    Regex,
    AddedToken, 
)
from tokenizers.normalizers import Normalizer
from transformers import AutoTokenizer
import re

# Normalizers

1. Unicode normalization 
2. Mapping from 'y' to 'i (bác sỹ -> bác sĩ) except when 'y' is alone or when it's in the compound 'uy' 

In [2]:
test = "thy"
test_pattern = re.compile(r"(?:[th])*y")

print(re.findall(test_pattern, test))

['thy']


In [3]:
pattern = re.compile(r"(?<=\b[hklmnst])(?:\S*)(?<=[^a\á\à\ạ\ả\ã\â\ấ\ầ\ậ\ẩ\ẫ\ă\ắ\ằ\ẳ\ẵ\ặu\ú\ù\ụ\ủ\ũ\s])([y\ỷ\ỹ\ỵ\ỳ\ý])\b")
test_text = "bác sỹ, thạc sỹ và ca sỹ, thý thoy mình bị suy dinh dưỡng."


print(re.findall(pattern, test_text))

['ỹ', 'ỹ', 'ỹ', 'ý', 'y']


In [4]:
from encodings import normalize_encoding


class CustomNormalizer: 
    def __init__(self): 
        self.pattern = re.compile(r"(?<=\b[hklmnst])(?:\S*)(?<=[^a\á\à\ạ\ả\ã\â\ấ\ầ\ậ\ẩ\ẫ\ă\ắ\ằ\ẳ\ẵ\ặu\ú\ù\ụ\ủ\ũ\s])([y\ỷ\ỹ\ỵ\ỳ\ý])\b")
        self.y_to_i_map = {
            'y': 'i',
            'ỷ': 'ỉ',
            'ỹ': 'ĩ',
            'ỵ': 'ị',
            'ỳ': 'ì',
            'ý': 'í'
        }

    def normalize(self, normalized: NormalizedString): 
        original = normalized.original
        matches = self.pattern.findall(original)

        for match in matches: 
            normalized.replace(match, self.y_to_i_map[match])

        print(normalized.normalized)
        normalized.nfc()

In [5]:
test_text = "bác sỹ, thạc sỹ và ca sỹ, thý thoy mình bị suy dinh dưỡng."
norm = NormalizedString(test_text)
norm.lowercase()

norm.normalized

'bác sỹ, thạc sỹ và ca sỹ, thý thoy mình bị suy dinh dưỡng.'

In [6]:
re.findall(re.compile(r"(?<=\b[hklmnst])(?:\S*)(?<=[^a\á\à\ạ\ả\ã\â\ấ\ầ\ậ\ẩ\ẫ\ă\ắ\ằ\ẳ\ẵ\ặu\ú\ù\ụ\ủ\ũ\s])(ý)\b"), test_text)

['ý']

In [7]:
tokenizer = Tokenizer(models.BPE(unk_token=None, fuse_unk=False, dropout=None, end_of_word_suffix="", continuing_subword_prefix="", byte_fallback=True,))

In [8]:
normalizer_sequence = normalizers.Sequence([
    normalizers.NFC(), 
])

# Pre-tokenizers


In [9]:
pretokenizer_sequence = pre_tokenizers.Sequence([
    pre_tokenizers.Split(Regex("(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"), behavior="isolated", invert=False),
    pre_tokenizers.ByteLevel(add_prefix_space=False, use_regex=False)
])

# Decoder

In [11]:
decoder = decoders.ByteLevel()

# Generate special tokens

In [12]:
cls = AddedToken(content="<CLS>", lstrip=False, normalized=False, rstrip=False, single_word=True)
pad = AddedToken(content="<PAD>", lstrip=False, normalized=False, rstrip=False, single_word=True)
mask = AddedToken(content="<MASK>", lstrip=False, normalized=False, rstrip=False, single_word=True)
sep = AddedToken(content="<SEP>", lstrip=False, normalized=False, rstrip=False, single_word=True)
unk = AddedToken(content="<UNK>", lstrip=False, normalized=False, rstrip=False, single_word=True)

# Create dataset generator

In [13]:
from datasets import load_dataset

raw_dataset = load_dataset("arrow", data_files="../data_all/data/data_00002.arrow")

In [14]:
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["text"]

In [15]:
training_corpus = get_training_corpus()

# Combine & Train

In [16]:
tokenizer.normalizer = normalizer_sequence
tokenizer.pre_tokenizer = pretokenizer_sequence
tokenizer.decoder = decoder

In [17]:
# (30000 + 261 + 64 - 53) 
vocab_size = 30272

In [18]:
trainer = trainers.BpeTrainer(vocab_size=500, show_progress=True, max_token_length=2048) 

In [19]:
tokenizer.train_from_iterator(training_corpus, trainer=trainer)

# Config tokenizer post processors

In [20]:
# add special tokens 

tokenizer.add_special_tokens([cls, pad, mask, sep, unk])

5

In [21]:
# enable padding 
tokenizer.enable_padding(pad_id=501, pad_type_id=501, pad_token="<PAD>")

In [22]:
# enable truncation 
tokenizer.enable_truncation(max_length=2048)

In [33]:
# set postprocessors
postprocessor_sequence = processors.Sequence([
    processors.ByteLevel(trim_offsets=False), 
    processors.TemplateProcessing(
    single="<CLS> $0 <SEP>",
    pair="<CLS> $A <SEP> $B:1 <SEP>:1",
    special_tokens=[("<CLS>", 15000), ("<SEP>", 15003)]),
])

tokenizer.post_processor = postprocessor_sequence

In [34]:
tokenizer

Tokenizer(version="1.0", truncation=TruncationParams(direction=Right, max_length=2048, strategy=LongestFirst, stride=0), padding=PaddingParams(strategy=BatchLongest, direction=Right, pad_to_multiple_of=None, pad_id=501, pad_type_id=501, pad_token="<PAD>"), added_tokens=[{"id":15000, "content":"<CLS>", "single_word":True, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":15001, "content":"<PAD>", "single_word":True, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":15002, "content":"<MASK>", "single_word":True, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":15003, "content":"<SEP>", "single_word":True, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":15004, "content":"<UNK>", "single_word":True, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=Sequence(normalizers=[NFC()]), pre_tokenizer=Sequence(pretokenizers=[Split(pattern=Regex("(?i:'s|'t|'re|'ve|'m|

In [32]:
tokenizer.token_to_id("<SEP>")

15003

# Examine after training

In [35]:
tokens = tokenizer.encode_batch([raw_dataset["train"][0]["text"], raw_dataset["train"][1]["text"]])

In [40]:
"".join([tokenizer.id_to_token(id) for id in tokens[1].ids])

'<CLS>NamĠsinhĠbá»ĭĠbáº¡nĠÄĳÃ¢mĠtrá»įngĠthÆ°Æ¡ngĊNamĠsinhĠbá»ĭĠbáº¡nĠÄĳÃ¢mĠtrá»įngĠthÆ°Æ¡ngĠtrongĠgiá»ĿĠraĠchÆ¡iĊChá»§ĠnháºŃt,Ġ05/03/2017Ġ16:53ĠPMĠGMT+7Ċ(VTCĠNews)Ġ-ĠXÃ´ĠxÃ¡tĠvá»ĽiĠnhÃ³mĠbáº¡nĠÄĳangĠÄĳÃ¹aĠnghá»ĭchĠtrÃªnĠsÃ¢nĠtrÆ°á»Ŀng,ĠDĠquayĠlÆ°ngĠláº¡iĠthÃ¬Ġbáº¥tĠngá»ĿĠbá»ĭĠbáº¡nĠÄĳÃ¢mĠtrá»įngĠthÆ°Æ¡ng.ĊNamĠsinhĠHÃłĠNá»ĻiĠÄĳÃ¢mĠbáº¡nĠcÃ¹ngĠlá»ĽpĠtá»ŃĠvongĊNgÃłyĠ5/3,ĠcÃ´ngĠanĠTPĠQuyĠNhÆ¡nĠchoĠbiáº¿t,ĠÄĳÆ¡nĠvá»ĭĠÄĳangĠtiáº¿nĠhÃłnhĠxÃ¡cĠminhĠvá»¥ĠemĠT.K.DĠ(SNĠ2003,Ġhá»įcĠsinhĠlá»ĽpĠ8)Ġbá»ĭĠbáº¡nĠÄĳÃ¢mĠtrá»įngĠthÆ°Æ¡ngĠtáº¡iĠtrÆ°á»Ŀng.ĊTheoĠthÃ´ngĠtinĠgiaĠÄĳÃ¬nh,Ġkhoáº£ngĠ15hĠngÃłyĠ2/3,ĠtrongĠgiá»ĿĠraĠchÆ¡i,ĠemĠT.K.DĠxáº£yĠraĠxÃ´ĠxÃ¡tĠvá»ĽiĠbáº¡nĠT.N.T.P.ĠNgayĠkhiĠDĠquayĠÄĳiĠthÃ¬Ġbáº¥tĠngá»ĿĠbá»ĭĠbáº¡nĠP.ĠdÃ¹ngĠdaoĠÄĳÃ¢mĠvÃłoĠlÆ°ngĠtrá»įngĠthÆ°Æ¡ng.ĠNgayĠláºŃpĠtá»©c,ĠDĠÄĳÆ°á»£cĠtháº§yĠcÃ´ĠgiÃ¡oĠÄĳÆ°aĠÄĳáº¿nĠbá»ĩnhĠviá»ĩnĠÄĲaĠkhoaĠTPĠQuyĠNhÆ¡nĠcáº¥pĠcá»©u.ĊAnhĠTráº§nĠNgá»įcĠDÅ©ngĠ(bá»ĳĠemĠD)ĠchoĠbiáº¿t,Ġhiá»ĩnĠtáº¡iĠDĠÄĳÃ£ĠquaĠcÆ¡nĠnguyĠká»ĭchĠnhÆ°ngĠváº«nĠpháº£iĠnáº±mĠviá»ĩnĠÄĳá»ĥĠtiáº¿p

In [27]:
decoded = tokenizer.decode_batch([token.ids for token in tokens])

In [28]:
decoded


# https://github.com/huggingface/tokenizers/issues/282

['Molypden lần đầu được khai thác ở đây từ cuối những năm 60 đến cuối những năm 70, nhưng dần thoái trào khi lợi nhuận giảm xuống. Đến cuối thập kỷ, giá đã tăng trở lại lần nữa khi các khu mỏ giàu molypden ở Alaska, British Columbia và miền tây nước Mỹ ...\nĐọc thêm\nNgôi nhà gạch ngập tràn hạnh phúc của vợ chồng trẻ và 2 ...\nNgôi nhà gạch ngập tràn hạnh phúc của vợ chồng trẻ và 2 cô con gái. Với mong muốn 2 cô con gái có môi trường vui chơi, phát triển toàn diện, đôi vợ chồng trẻ đã quyết định xây dựng ngôi nhà mới theo sở thích của 2 cô bé. Tên của 2 cô con gái Chi và Vi, gộp lại và ...\nĐọc thêm\nCÁC BÀI SUY NIỆM CHÚA NHẬT 16 THƯỜNG NIÊN. NĂM ...\n2019-7-19\u2002·\u2002Lời Chúa hôm nay tiếp tục khai triển đề tài này, đặc biệt qua câu chuyện Chúa đến thăm chị em cô Mác-ta và Ma-ri-a tại nhà các cô. Để giúp chúng ta biết quý trọng sự viếng thăm của Chúa, Phụng vụ Lời Chúa lấy lại câu chuyện Đức Chúa ghé thăm ông Áp …\nĐọc thêm\nthiết bị nghiền simex\nGiới thiệu chung Công ty NGUYỄN V